In [1]:
from argparse import Namespace
import torch
import torch.nn as nn
import torchvision.models as models
from get_dataset import GetTransformedDataset
from simclr import simclr_framework


class ResNet18(nn.Module):
    def __init__(self, out_dim):
        super(ResNet18, self).__init__()
        self.backbone = self._get_basemodel(out_dim)
        dim_mlp = self.backbone.fc.in_features
        self.backbone.fc = nn.Sequential(
            nn.Linear(dim_mlp, dim_mlp), nn.ReLU(), self.backbone.fc)

    def _get_basemodel(self, out_dim):
        return models.resnet18(weights=None, num_classes=out_dim)

    def forward(self, x):
        return self.backbone(x)

In [2]:
def train(args):

    print(args.device)

    dataset = GetTransformedDataset()
    train_dataset = dataset.get_tiny_imagenet_train(args.n_views)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=args.workers, pin_memory=True, drop_last=True)

    model = ResNet18(out_dim=args.out_dim)

    optimizer = torch.optim.Adam(model.parameters(), args.lr, weight_decay=args.weight_decay)

    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader), eta_min=0, last_epoch=-1)
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: 1.0)
 
    simclr = simclr_framework(model=model, optimizer=optimizer, scheduler=scheduler, args=args)

    print("="*50)
    print(f"Trining SimCLR_TIN_lr{args.lr}_wd{args.weight_decay}_temperature{args.temperature}_bt{args.batch_size}_e{args.epochs}")
    print("="*50)
    print('training started..')
    simclr.train(train_loader) 
    print('training completed..')


In [ ]:
temp_list = [0.07]
lr_list = [1e-3]
wd_list = [0]
for temp in temp_list:
    for lr in lr_list:
        for wd in wd_list:
            args = Namespace
            # Hyperparameters
            args.batch_size = 512
            args.epochs = 500
            args.lr = lr
            args.temperature = temp
            args.weight_decay = wd
            # Other settings
            args.fp16_precision = False
            args.device = torch.device('cuda')
            args.gpu_index = 0
            args.log_every_n_steps = 1
            args.n_views = 2
            args.out_dim = 100
            args.seed = 1
            args.workers = 4
            args.log_dir = f"/root/Lab3-1/logs/simclr_TIN_lr{args.lr}_wd{args.weight_decay}_temp{args.temperature}_bt{args.batch_size}_e{args.epochs}"
            train(args)

cuda
Trining SimCLR_TIN_lr0.001_wd0_temperature0.07_bt512_e500
training started..


  0%|          | 0/195 [00:00<?, ?it/s]/usr/local/miniconda3/envs/CV/lib/python3.8/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
 78%|███████▊  | 153/195 [01:42<00:24,  1.69it/s]